In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from sentence_transformers import SentenceTransformer

c:\Users\Administrator\anaconda3\envs\hrbots\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
from langchain.llms import HuggingFaceHub

In [4]:

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [5]:
model.save("D:\HR-bot\HR-BOT\models\embedding")

In [6]:
PINECONE_API_KEY = "765789ec-4ce4-4858-be09-31128f07c5f5"
PINECONE_API_ENV = "gcp-starter"

In [7]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [8]:
extracted_data = load_pdf(r"D:\kanini-hackathon\hr-bot\HR-BOT\data")

In [9]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 72


In [11]:
text_chunks[0]

Document(page_content='Table of Contents  \n1. Office Decorum  \n1.1 Office Timing  \n1.2 Leave Policy                                             \n1.3 Dress Code  \n1.4 Cleanliness  \n1.5 Work from home policy  \n1.6 ID Card / Access card  \n1.7 Unauthorized Access  \n2. Disaster recovery P olicy    \n3. Clean Desk & Clear screen Policy    \n4. Open Door Policy                                                \n5. Confidentiality and security policy                    \n5.1 Company and Personnel Information', metadata={'source': 'D:\\kanini-hackathon\\hr-bot\\HR-BOT\\data\\document.pdf', 'page': 0})

In [12]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="D:\HR-bot\HR-BOT\models\embedding")
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()

In [14]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="hr-bot"

In [31]:


#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings,metadatas=[{'pdf_name': p.metadata["source"].split("\\")[-1].lower()} for p in text_chunks], index_name=index_name)

In [15]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "tell me about dress code ?"

docs=docsearch.similarity_search(query, k=10)

print("Result", docs)

Result [Document(page_content='• Earned leave will not be compensated for notice period.  \n \nH. Trainees  \n• One leave per month can be availed during the training period.  \n \nI. Late In/ Early out with Prior Permission  \nEmployee s can avail monthly two permissions ie. Two hours each.  Exceeding two hours \nwould be considered as half day leave/month.  \n \n1.3  Dress Code  \nAll employees are expected to be properly groom ed and wear proper formal dress.   \n• Mon -Thur – Formal wear / KANINI T-shirts  \n• Friday – Casuals (No shots(3/4ths), No torn jeans, No skirts, No sleeveless) + \nKANINI T -shirts  \n1.4 Cleanliness  \nEmployees are expected to keep their surroundings neat, clean, and tidy. Use \nprovided dustbins to dispose of any wastes. Each employee must take responsibility for \nthe workplace, dining hall, and of the office in general.  \n \n1.5 Work from home policy  \nA. Policy Statement  \nWork at home was brought in for the situation where employee has to complete

In [16]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [17]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [18]:
llm=CTransformers(model=r"D:\kanini-hackathon\hr-bot\HR-BOT\models\llm\llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={
                          'temperature':0.8})

In [19]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [20]:
user_input="how many days employee need to come office ?"
result=qa({"query": user_input})
print("Response : ", result["result"])

Number of tokens (615) exceeded maximum context length (512).
Number of tokens (616) exceeded maximum context length (512).
Number of tokens (617) exceeded maximum context length (512).
Number of tokens (618) exceeded maximum context length (512).
Number of tokens (619) exceeded maximum context length (512).
Number of tokens (620) exceeded maximum context length (512).
Number of tokens (621) exceeded maximum context length (512).
Number of tokens (622) exceeded maximum context length (512).
Number of tokens (623) exceeded maximum context length (512).
Number of tokens (624) exceeded maximum context length (512).
Number of tokens (625) exceeded maximum context length (512).
Number of tokens (626) exceeded maximum context length (512).
Number of tokens (627) exceeded maximum context length (512).
Number of tokens (628) exceeded maximum context length (512).
Number of tokens (629) exceeded maximum context length (512).
Number of tokens (630) exceeded maximum context length (512).
Number o

KeyboardInterrupt: 